In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym

In [2]:
#애니메이션을 만드는 함수
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
    
    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval = 50)
    writergif = animation.PillowWriter(fps=30) 
    anim.save("movie_cartpole.gif",writer=writergif)
    display(display_animation(anim, default_mode='loop'))

In [3]:
#수레를 무작위로 움직임
frames = []
env = gym.make('CartPole-v0')
observation = env.reset() #환경 초기화

for step in range(0,200):
    frames.append(env.render(mode='rgb_array')) #v프레임에 각 시각의 이미지 추가
    action = np.random.choice(2) #0-수레를 왼쪽으로, 1-수레를 오른쪽으로
    observation, reward, done, info = env.step(action) #action실행
    
#애니메이션파일 재생
display_frames_as_gif(frames)

C:\Users\user\anaconda3\envs\rl_env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\Users\user\anaconda3\envs\rl_env\lib\site-packages\JSAnimation\html_writer.py:282: MatplotlibDeprecationWarning: 
The 'clear_temp' parameter of setup() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. If any parameter follows 'clear_temp', they should be passed as keyword, not positionally.
  frame_prefix, clear_temp=False)


AttributeError: 'HTMLWriter' object has no attribute '_temp_names'

### 상태의 이산변수 변환

In [5]:
#상수 정의
ENV = 'CartPole-v0' #태스크 이름
NUM_DIZITIZED = 6 #각 상태를 이산변수로 변환할 구간 수

#CartPole 실행
env = gym.make(ENV) #실행할 태스크 선택
observation = env.reset() #환경 초기화

#이산값으로 만들 구간 계산
def bins(clip_min, clip_max, num): #관측된 상태(연속값)를 이산값으로 변환하는 구간 계산
    return np.linspace(clips_min, clips_max, num+1)[1:-1]

In [8]:
#관측된 상태를 이산값으로 변환
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    #순서대로 수레의 위치, 수레의 속도, 봉의 각도, 봉의 각속도
    digitized = [
        np.digitize(cart_pos, bins = bins(-2.4, 2.4, NUM_DIZITIZED)),
        np.digitize(cart_v, bins = bins(-3.0, 3.0, NUM_DIZITIZED)),
        np.digitize(cart_pos, bins = bins(-0.5, 0.5, NUM_DIZITIZED)),
        np.digitize(cart_pos, bins = bins(-2.0, 2.0, NUM_DIZITIZED))]
    return sum([x*(NUM_DIZITIZED**i)for i, x in enumerate(digitized)])

### Q-learning 구현
with 3 classes
- Agent: 수레, 'Q함수 수정', '다음 행동 결정'
- Brain: '수레가 관찰한 상태를 이산변수로 변환', 'Q테이블 수정', 'Q테이블을 이용해 행동 결정'
- Environment: OpenAI Gyem이 실행되는 실행환경, 'run'

#### (1) agent가 observation_t를 Brain에 전달
#### (2) Brain은 전달받은 현재 상태를 이산변수로 변환하고, Q테이블을 참조해서 행동을 결정해서, agent에게 행동 전달
#### (3) agent는 environment에 행동action_t를 전달하고 
#### (4) Environment는 다음 상태 observation_t+1과 즉각보상 reward_t+1을 agent에게 반환
#### (5) agent가 전달한, 현재상태, 행동, 다음 상태, 즉각보상 (transition) 정보를 바탕으로, Brain은 Q테이블 수정